# Diffusion

In [3]:
import numpy as np
import scipy 
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline

### **Microscopic Aspects of Diffusion and Generating Functions**  

Einstein’s theory of diffusion is based on **random walk models**, leading to a fundamental equation that relates the **mean square displacement (MSD)** of a particle in $ d $ dimensions to time. We explore this by simulating $ N $ random walks with $ n $ steps, all starting from the origin $ r_0 = 0 $:  

$$
R_n = \sum^{n}_{i=0} r_i
$$

### **Mean Square Displacement and Diffusion Coefficient**  

Expressing the number of steps in terms of time increments $ n = \frac{t}{\delta t} $, we compute the **ensemble average** over $ N $ random walkers in $ d = 1,2,3 $ dimensions:

$$
\langle R^2_n\rangle = \sum_i \sum_j \langle r_i r_j \rangle = \sum_i \langle r^2_i \rangle
$$

Assuming independent steps, we get:

$$
\langle R^2_n\rangle = \sum_i d \cdot \langle \delta x^2_i \rangle = d \cdot n \cdot  \delta x^2
$$

$$
\langle R^2_n\rangle = d \cdot \frac{t}{\delta t}  \cdot \delta x^2
$$

Grouping constants together, we define the **diffusion coefficient** $ D $, which characterizes the spreading of a particle:

$$
D = \frac{\langle \delta x^2 \rangle}{2\delta t}
$$

This leads to the **general expression** for the **mean square displacement (MSD)** as a function of time:

$$
\langle R^2 (t) \rangle = 2d D t
$$

Any motion that follows this **linear scaling with time** is called **diffusive**.

In [ ]:
d=3 
t= np.linspace(0, 1, 10000)

for D in [0.01, 0.1, 1, 10]:
    
    plt.plot(t, 2*d *D * t**0.5)
    #plt.loglog(t, 2*d *D * t**0.5)

plt.ylabel('$MSD(t)$')
plt.xlabel('$t$')

:::{admonition} **Derivation of the Diffusion Equation from a Random Walk via Recurrence Relations**  
:class: tip, dropdown

To derive the **diffusion equation** from a **random walk**, we consider a discrete-time process where a particle moves in steps of size $ \delta x $ at each time increment $ \delta t $. The probability $ P(x, t) $ of finding the particle at position $ x $ at time $ t $ satisfies a **recurrence relation**.

**Writing the Recurrence Relation for a Symmetric Random Walk**  

At each time step, the particle moves:
- **Left** with probability $ p = 1/2 $,
- **Right** with probability $ q = 1/2 $.

Thus, the probability at time $ t $ is given by the sum of probabilities from the previous step:

$$
P(x, t) = \frac{1}{2} P(x - \delta x, t - \delta t) + \frac{1}{2} P(x + \delta x, t - \delta t).
$$

Expanding $ P(x, t) $ using a **Taylor series** in small $ \delta x $ and $ \delta t $:

$$
P(x \pm \delta x, t - \delta t) \approx P(x, t - \delta t) \pm \delta x \frac{\partial P}{\partial x} + \frac{(\delta x)^2}{2} \frac{\partial^2 P}{\partial x^2}.
$$

Substituting into the recurrence relation:

$$
P(x, t) = \frac{1}{2} \left[ P(x, t - \delta t) - \delta x \frac{\partial P}{\partial x} + \frac{(\delta x)^2}{2} \frac{\partial^2 P}{\partial x^2} \right] + \frac{1}{2} \left[ P(x, t - \delta t) + \delta x \frac{\partial P}{\partial x} + \frac{(\delta x)^2}{2} \frac{\partial^2 P}{\partial x^2} \right].
$$

Simplifying:

$$
P(x, t) = P(x, t - \delta t) + \frac{(\delta x)^2}{2} \frac{\partial^2 P}{\partial x^2}.
$$

Rearranging,

$$
\frac{P(x, t) - P(x, t - \delta t)}{\delta t} = \frac{(\delta x)^2}{2 \delta t} \frac{\partial^2 P}{\partial x^2}.
$$

Taking the limit $ \delta t \to 0 $ and defining the **diffusion coefficient** $ D = \frac{(\delta x)^2}{2 \delta t} $, we obtain the **diffusion equation**:

$$
\frac{\partial P(x,t)}{\partial t} = D \frac{\partial^2 P(x,t)}{\partial x^2}.
$$

:::


In [ ]:
# Parameters
num_walkers = 10000  # Number of random walkers
num_steps = 500  # Number of steps
delta_x = 1  # Step size
delta_t = 1  # Time step
D = (delta_x**2) / (2 * delta_t)  # Diffusion coefficient

# Initialize positions
positions = np.zeros(num_walkers)

# Store probability distribution at different time steps
time_snapshots = [10, 100, 500]
histories = {t: None for t in time_snapshots}

# Perform random walk
for step in range(num_steps):
    positions += np.random.choice([-delta_x, delta_x], size=num_walkers)  # Step left or right
    if step + 1 in time_snapshots:
        hist, bin_edges = np.histogram(positions, bins=np.arange(-50, 51, 1), density=True)
        histories[step + 1] = (bin_edges[:-1], hist)

# Compute analytical Gaussian solution
x = np.linspace(-50, 50, 200)
fig, ax = plt.subplots(figsize=(8, 5))

for t in time_snapshots:
    # Simulated random walk distribution
    ax.plot(histories[t][0], histories[t][1], 'o', label=f"Random Walk at t={t}", alpha=0.6)

    # Diffusion equation Gaussian solution
    sigma_t = np.sqrt(2 * D * t)
    gaussian_solution = (1 / np.sqrt(4 * np.pi * D * t)) * np.exp(-x**2 / (4 * D * t))
    ax.plot(x, gaussian_solution, '-', label=f"Diffusion Eq. at t={t}")

# Formatting the plot
ax.set_title("Random Walk vs. Diffusion Equation", fontsize=14)
ax.set_xlabel("Position (x)", fontsize=12)
ax.set_ylabel("Probability Density $\\rho(x,t)$", fontsize=12)
ax.legend()
ax.grid(alpha=0.3)
plt.show()


:::{admonition} **Generating Functions and the Emergence of the Gaussian Distribution**  
:class: tip, dropdown

To formally derive the **Gaussian nature** of the probability distribution for the random walk, we start with the **definition of the probability distribution** of the sum of independent steps:

$$
P_n(R) = \int dx_1 dx_2 \dots dx_n \, P(x_1) P(x_2) \dots P(x_n) \, \delta\left(R - \sum_{i=1}^{n} x_i \right)
$$

Here, $ P(x) $ represents the probability density function (PDF) for a single step. The presence of the **Dirac delta function** enforces the sum constraint.

We introduce the **Fourier representation** of the delta function:

$$
\delta(R - \sum_i x_i) = \int \frac{dk}{2\pi} e^{-ik(R - \sum_i x_i)}
$$

Substituting into $ P_n(R) $, we obtain:

$$
P_n(R) = \int \frac{dk}{2\pi} e^{-ikR} \left[ \int dx \, P(x) e^{ikx} \right]^n
$$

The term in brackets is the **Generating function** (Fourier transform of the step distribution):

$$
G(k) = \int dx \, P(x) e^{ikx}
$$

Since each step is independent, the generating function of the sum factorizes:

$$
P_n(R) = \int \frac{dk}{2\pi} e^{-ikR} \left[G(k)\right]^n
$$

For small steps, we expand $ G(k) $ in a Taylor series:

$$
G(k) = 1 - \frac{1}{2} k^2 \langle x^2 \rangle + \mathcal{O}(k^4)
$$

Thus, for large $ n $, we approximate:

$$
\left[G(k)\right]^n \approx e^{ -\frac{1}{2} n k^2 \langle x^2 \rangle }
$$

Taking the inverse Fourier transform, we obtain the **Gaussian probability density function (PDF)**:

$$
P_n(R) = \frac{1}{\sqrt{2\pi n \langle x^2 \rangle}} e^{ -\frac{R^2}{2n \langle x^2 \rangle} }
$$

Since $ n = \frac{t}{\delta t} $, this gives the **diffusion propagator**:

$$
P(R, t) = \frac{1}{\sqrt{4\pi D t}} e^{-R^2 / (4Dt)}
$$

This result shows that the **probability of finding the particle at position $ R $ at time $ t $ follows a Gaussian distribution**, which is the solution to the **diffusion equation**:

$$
\frac{\partial P}{\partial t} = D \nabla^2 P
$$

Thus, diffusion emerges **naturally from the sum of many independent random steps**, justifying the **Gaussian approximation** via the central limit theorem.

:::


### **Summary of Key Results**

1. **Mean Square Displacement (MSD)**:  
   $$
   \langle R^2 (t) \rangle = 2d D t
   $$

2. **Diffusion Coefficient**:  
   $$
   D = \frac{\langle \delta x^2 \rangle}{2\delta t}
   $$

3. **Probability Density Function for a Random Walker**:  
   $$
   P(R, t) = \frac{1}{\sqrt{4\pi D t}} e^{-R^2 / (4Dt)}
   $$

### **Brownian Motion**  

:::{figure-md} markdown-fig  
<img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Brownian_motion_large.gif" alt="Brownian Motion Animation" style="width:10%">  

**Animation of Brownian Motion**  
:::  

- **Brownian motion** describes the random movement of a particle suspended in a solvent composed of much smaller molecules. This motion arises from **a large number of independent random collisions** with solvent molecules. By invoking the **Central Limit Theorem (CLT)**, we approximate the displacement of the particle over a small time step $ dt $ as a normally distributed random variable:

$$
x(t+dt) - x(t) \sim \mathcal{N}(0, \sqrt{2D dt})
$$

- If the particle starts at position $ \mu = 0 $, the variance of its position evolves as:

$$
\sigma^2 = 2Dt
$$

where $ D $ is the **diffusion coefficient**, which is related to the step size and time increments of the **discrete random walk model**, as derived in the lecture.

- We can express Brownian motion in an iterative form:

$$
x(t+dt) = x(t) + \sqrt{2D dt} \cdot N(0,1)
$$

This formulation highlights the connection between Brownian motion and Gaussian distributions. Specifically, we rewrite the update step using the general form of a normally distributed random variable:

$$
N(\mu, \sigma^2) = \mu + \sigma N(0,1)
$$

where the mean $ \mu = x(t) $ and the standard deviation $ \sigma = \sqrt{2D dt} $, showing how each step is **normally distributed and accumulates over time**, leading to diffusion-like behavior.

In [6]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
num_steps = 1000  # Number of steps
num_walks = 5  # Number of random walks to visualize
step_size = 1  # Step size
dt = 1  # Time step

# 1D Brownian Motion
fig, ax1 = plt.subplots(figsize=(10, 5))

for _ in range(num_walks):
    x = np.cumsum(np.random.normal(loc=0, scale=np.sqrt(dt) * step_size, size=num_steps))  # 1D Random Walk
    ax1.plot(np.arange(num_steps), x, alpha=0.7)

ax1.set_title("1D Brownian Motion", fontsize=14)
ax1.set_xlabel("Time Step", fontsize=12)
ax1.set_ylabel("Position", fontsize=12)
ax1.grid(alpha=0.3)

# 2D Brownian Motion
fig, ax2 = plt.subplots(figsize=(6, 6))

for _ in range(num_walks):
    x = np.cumsum(np.random.normal(loc=0, scale=np.sqrt(dt) * step_size, size=num_steps))  # x displacement
    y = np.cumsum(np.random.normal(loc=0, scale=np.sqrt(dt) * step_size, size=num_steps))  # y displacement
    ax2.plot(x, y, alpha=0.7)

ax2.set_title("2D Brownian Motion", fontsize=14)
ax2.set_xlabel("X Position", fontsize=12)
ax2.set_ylabel("Y Position", fontsize=12)
ax2.grid(alpha=0.3)

# Show plots
plt.show()


### **Diffusion Equation and 1D Evolution Animation**  

- The probability distribution of individual **random walkers**, denoted as $ \rho({\bf r},t) $, evolves over time according to the **diffusion equation**:

$$
\frac{\partial\rho ({\bf r}, t)}{\partial t} = D\nabla^2\rho({\bf r}, t)
$$

This equation, first formulated empirically as **[Fick's laws](https://en.wikipedia.org/wiki/Fick%27s_laws_of_diffusion)**, describes the spreading of a probability distribution due to random motion. Examples of diffusion include:
  - The dispersion of dye in a liquid.
  - The spreading of perfume molecules in a room.

- The **diffusion equation is a second-order partial differential equation (PDE)**. Unlike Newton’s or Schrödinger’s equations, it exhibits **irreversible** behavior, representing how probability "spreads" over time. The **diffusion coefficient** $ D $ has units of $ [L^2]/[T] $, indicating the rate of spreading.

- For the case of **free diffusion**, the equation has an **exact analytical solution** in one dimension. More complex **reaction-diffusion systems** generally require numerical solutions using finite difference approximations.

- A key special case is **free diffusion in 1D**, where the probability distribution follows a Gaussian profile with a time-dependent **mean square displacement (MSD)**:

$$
\sigma(t) = \sqrt{2D t}
$$

Substituting this into the diffusion equation gives the **fundamental solution**:

$$
\rho(x,t) = \frac{1}{\sqrt{2\pi \sigma(t)^2}} \exp\left(-\frac{x^2}{2\sigma(t)^2}\right).
$$

This is the **spreading Gaussian solution**, showing how an initial **delta function** (localized distribution) broadens over time.


:::{admonition} **Solution of the 1D Diffusion Equation via Generating Functions**  
:class: tip, collapse

We aim to solve the **1D diffusion equation**:

$$
\frac{\partial \rho(x,t)}{\partial t} = D \frac{\partial^2 \rho(x,t)}{\partial x^2}
$$

using **generating functions**.



**Step 1: Defining the Generating Function**  
The **generating function** (or **Fourier transform**) of $ \rho(x,t) $ is:

$$
G(k,t) = \int_{-\infty}^{\infty} e^{ikx} \rho(x,t) dx
$$

Applying the Fourier transform to both sides of the diffusion equation:

$$
\frac{\partial}{\partial t} G(k,t) = D (-k^2) G(k,t)
$$

which simplifies to a simple **first-order differential equation**:

$$
\frac{\partial G(k,t)}{\partial t} = -D k^2 G(k,t)
$$



**Step 2: Solving the ODE for $ G(k,t) $**  
This equation has the **exponential solution**:

$$
G(k,t) = G(k,0) e^{-D k^2 t}
$$

For an initial **delta function** distribution $ \rho(x,0) = \delta(x) $, its Fourier transform is **unity**:

$$
G(k,0) = 1
$$

Thus, the **evolved generating function** is:

$$
G(k,t) = e^{-D k^2 t}
$$



**Step 3: Inverting the Fourier Transform**  
To find $ \rho(x,t) $, we take the **inverse Fourier transform**:

$$
\rho(x,t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} e^{-D k^2 t} e^{-ikx} dk
$$

This integral is a well-known **Gaussian integral**, yielding:

$$
\rho(x,t) = \frac{1}{\sqrt{4\pi D t}} e^{-x^2 / (4Dt)}
$$

This is the **fundamental solution** of the 1D diffusion equation, showing that the initial **delta function** spreads into a **Gaussian distribution** with variance:

$$
\sigma^2 = 2Dt
$$

:::

### **Visualization: 1D Diffusion Evolution**

Below, we numerically compute and plot the spreading Gaussian solution

The plot illustrates the **evolution of the 1D diffusion equation** solution, showing how an initially **localized delta function** spreads into a **Gaussian distribution** over time.

- At **small times ($ t = 0.1 $)**, the distribution is sharply peaked.
- As **time increases**, the probability density **broadens**, following the expected **Gaussian profile** with variance $ \sigma^2 = 2Dt $.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Parameters
D = 1  # Diffusion coefficient
x_min, x_max = -10, 10  # Spatial range
num_points = 200  # Number of spatial points
num_frames = 100  # Number of time steps
dt = 0.1  # Time step

# Discretized space
x = np.linspace(x_min, x_max, num_points)
sigma_t = lambda t: np.sqrt(2 * D * t)  # Standard deviation of the Gaussian

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 5))
line, = ax.plot([], [], 'b-', lw=2)
ax.set_xlim(x_min, x_max)
ax.set_ylim(0, 0.5)
ax.set_xlabel("Position (x)", fontsize=12)
ax.set_ylabel("Probability Density $\\rho(x,t)$", fontsize=12)
ax.set_title("Evolution of 1D Diffusion", fontsize=14)
ax.grid(alpha=0.3)

# Initialize function
def init():
    line.set_data([], [])
    return line,

# Update function for animation
def update(frame):
    t = frame * dt + 0.1  # Avoid division by zero at t=0
    rho_x_t = (1 / np.sqrt(2 * np.pi * sigma_t(t)**2)) * np.exp(-x**2 / (2 * sigma_t(t)**2))
    line.set_data(x, rho_x_t)
    return line,

# Create animation
ani = animation.FuncAnimation(fig, update, frames=num_frames, init_func=init, blit=True, interval=50)
# Convert animation to HTML
html_anim = HTML(ani.to_jshtml())
plt.close(fig)  # Prevent duplicate display

# Display the animation
html_anim

### Numerical Solution of 2D diffusion equation using the finite difference method

- The probability density $\rho(x,y,t)$ is initialized as a **delta function** at the center.
- We use an **explicit finite difference scheme**, updating each grid point using the standard **5-point stencil**:

$$
  \rho_{i,j}^{t+dt} = \rho_{i,j}^t + D \frac{dt}{dx^2} \left( \rho_{i+1,j} + \rho_{i-1,j} + \rho_{i,j+1} + \rho_{i,j-1} - 4\rho_{i,j} \right)
$$

- The stability condition $dt < dx^2 / (4D)$ ensures numerical accuracy.

The final heatmap visualization shows the **spread of the probability distribution** over time due to diffusion. Let me know if you need further enhancements!

In [ ]:
# Parameters
time_steps = [0, 100, 250, 500]  # Selected time steps for snapshots

# Initialize grid with an initial delta function at the center
rho = np.zeros((L, L))
rho[L//2, L//2] = 1.0 / (dx**2)  # Initial peak

# Prepare figure
fig, axes = plt.subplots(1, len(time_steps), figsize=(15, 5))

# Finite difference loop (explicit scheme) with snapshots
snapshots = {}
for step in range(num_steps + 1):
    rho_new = rho.copy()
    rho_new[1:-1, 1:-1] = rho[1:-1, 1:-1] + D * dt / dx**2 * (
        rho[:-2, 1:-1] + rho[2:, 1:-1] + rho[1:-1, :-2] + rho[1:-1, 2:] - 4 * rho[1:-1, 1:-1]
    )
    rho = rho_new

    if step in time_steps:
        snapshots[step] = rho.copy()

# Plot snapshots
for ax, step in zip(axes, time_steps):
    im = ax.imshow(snapshots[step], extent=[-L//2, L//2, -L//2, L//2], cmap="hot", origin="lower")
    ax.set_title(f"t = {step * dt:.1f}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")

# Add colorbar
fig.colorbar(im, ax=axes.ravel().tolist(), label="Probability Density $\\rho(x, y, t)$")
plt.suptitle("Time Evolution of 2D Diffusion", fontsize=14)
plt.show()